In [1]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.plasma.standard_plasmas import assemble_plasma
from tardis.model import Radial1DModel
from tardis.io.atom_data import AtomData
from tardis.visualization.widgets.grotrian import GrotrianWidget
from tardis.io.config_internal import get_data_dir
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import os

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
### Run Simulation
config = Configuration.from_yaml('../../../docs/tardis_example.yml')
atom_data = AtomData.from_hdf(os.path.join(get_data_dir(), 'kurucz_cd23_chianti_H_He.h5'))
model = Radial1DModel.from_config(config, atom_data=atom_data)
plasma = assemble_plasma(config, model, atom_data=atom_data)
sim = Simulation.from_config(config, model=model, plasma=plasma)
sim.run()

Abundances have not been normalized to 1. - normalizing
Zeta_data missing - replaced with 1s. Missing ions: [(12, 13), (14, 15), (16, 17), (18, 19), (20, 21)]
/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

divide by zero encountered in true_divide

/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

invalid value encountered in true_divide

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
Zeta_data missing - replaced with 1s. Missing ions: [(12, 13), (14, 15), (16, 17), (18, 19), (20, 21)]
/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

divide by zero encountered in true_divide

/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

invalid value encountered in true_divide



TqdmHBox(children=(HTML(value='Iterations:', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

TqdmHBox(children=(HTML(value='Packets:   ', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

Shell No.,t_rad,next_t_rad,w,next_w
0,9.93e+03,1.01e+04,0.4,0.525
5,9.85e+03,1.03e+04,0.211,0.196
10,9.78e+03,1.02e+04,0.143,0.115
15,9.71e+03,9.88e+03,0.105,0.0843


/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

divide by zero encountered in true_divide

/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

invalid value encountered in true_divide



...

Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.48,0.486
5,1.16e+04,1.15e+04,0.174,0.18
10,1.13e+04,1.12e+04,0.105,0.108
15,1.09e+04,1.09e+04,0.0789,0.0793


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.486,0.486
5,1.15e+04,1.15e+04,0.18,0.177
10,1.12e+04,1.13e+04,0.108,0.107
15,1.09e+04,1.09e+04,0.0793,0.0811


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.486,0.483
5,1.15e+04,1.16e+04,0.177,0.17
10,1.13e+04,1.13e+04,0.107,0.107
15,1.09e+04,1.09e+04,0.0811,0.0799


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.483,0.482
5,1.16e+04,1.16e+04,0.17,0.172
10,1.13e+04,1.13e+04,0.107,0.105
15,1.09e+04,1.09e+04,0.0799,0.0807


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.482,0.478
5,1.16e+04,1.14e+04,0.172,0.177
10,1.13e+04,1.13e+04,0.105,0.107
15,1.09e+04,1.08e+04,0.0807,0.0814


In [3]:
diag = GrotrianWidget.from_simulation(sim)
diag.display()

/Users/archil/Documents/tardis_ayushi/tardis/visualization/widgets/grotrian.py:99: PerformanceWarning:

indexing past lexsort depth may impact performance.

/Users/archil/Documents/tardis_ayushi/tardis/visualization/widgets/grotrian.py:104: PerformanceWarning:

indexing past lexsort depth may impact performance.



FigureWidget({
    'data': [{'hovertemplate': 'Energy: 19.8 eV<br>Population: 0.20<extra></extra>',
              'line': {'color': 'black', 'width': 4.0},
              'mode': 'lines+text',
              'showlegend': False,
              'type': 'scatter',
              'uid': '3e6ebff9-afcf-4cd6-8b9b-60db931bf3ea',
              'x': [0, 1],
              'y': [1, 1]},
             {'hovertemplate': 'Energy: 20.6 eV<br>Population: 0.03<extra></extra>',
              'line': {'color': 'black', 'width': 1.0},
              'mode': 'lines+text',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'f57a3e12-6a16-4674-9f0e-e4f6117df9ef',
              'x': [0, 1],
              'y': [2, 2]},
             {'hovertemplate': 'Energy: 21.0 eV<br>Population: 0.18<extra></extra>',
              'line': {'color': 'black', 'width': 3.886147641627425},
              'mode': 'lines+text',
              'showlegend': False,
              'type': 'scatter',
    

In [4]:
diag = GrotrianWidget.from_simulation(sim)
diag.atomic_number = 8
diag.ion_number = 0
diag.display()

FigureWidget({
    'data': [{'hovertemplate': 'Energy: 0.0 eV<br>Population: 39.71<extra></extra>',
              'line': {'color': 'black', 'width': 4.0},
              'mode': 'lines+text',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'be225b25-4fa2-4562-b4ee-e82d4053a9f9',
              'x': [0, 1],
              'y': [1, 1]},
             {'hovertemplate': 'Energy: 2.0 eV<br>Population: 6.73<extra></extra>',
              'line': {'color': 'black', 'width': 3.5388805608331086},
              'mode': 'lines+text',
              'showlegend': False,
              'type': 'scatter',
              'uid': '46ae22ec-54af-471e-9fbb-8a56595cd5e6',
              'x': [0, 1],
              'y': [2, 2]},
             {'hovertemplate': 'Energy: 4.2 eV<br>Population: 0.14<extra></extra>',
              'line': {'color': 'black', 'width': 2.5277328375747725},
              'mode': 'lines+text',
              'showlegend': False,
              'type': '

In [5]:
# When number of electrons are very less (see atomic number 16),
# then just render an empty Grotrian diagram (or don't show it in options)

